<a href="https://colab.research.google.com/github/davis689/binder/blob/master/Keq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [3]:
import numpy as np
from sympy import *
from scipy.optimize import brentq,fsolve
import matplotlib.pyplot as plt
import requests
import lxml.html as lh
import pandas as pd
from functools import reduce


In [4]:
T,V,A,B,p,x,xi=var('T, V,A,B,p,x,xi')
R=8.3144598 # gas constant

Import data from webbook.nist.gov for each molecule

In [5]:
url = 'https://webbook.nist.gov/cgi/cbook.cgi?ID=C10024972&Units=SI&Mask=1#Thermo-Gas' #N2O
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-3] # change index to get different table. -1 means start at end
So=[]
So.append(float(df.iloc[2,1]))
df=df_list[-2]
tmp=df.iloc[1:9,1].values.tolist()
coef=[]
coef.append([float(tmp[i]) for i in range(len(tmp))])

In [6]:
url = 'https://webbook.nist.gov/cgi/cbook.cgi?ID=C7782447&Units=SI&Mask=1#Thermo-Gas' #O2
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-3] # change index to get different table. -1 means start at end
So.append(float(df.iloc[1,1]))
df=df_list[-2]
tmp=df.iloc[1:9,1].values.tolist()
coef.append([float(tmp[i]) for i in range(len(tmp))])

In [7]:
url = 'https://webbook.nist.gov/cgi/cbook.cgi?ID=C10102440&Units=SI&Mask=1#Thermo-Gas' #NO2
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-3] # change index to get different table. -1 means start at end
So.append(float(df.iloc[2,1]))
df=df_list[-2]
tmp=df.iloc[1:9,1].values.tolist()
coef.append([float(tmp[i]) for i in range(len(tmp))])

In [8]:
stoic=[-2,-3,4] # stoichiometry with reactant negative.
species=['N2O','O2','NO2']

# Input T and p and initial moles

In [9]:
#@title Input the temperature and pressure at which you want to calculate entropy change { display-mode: "form" }
pi = 1
pf =  .1#@param {type:"number"}
Ti = 298
Tf =  450#@param {type:"number"}
n_N2O =  1#@param {type:"number"}
n_O2 =  1#@param {type:"number"}
n_NO2 =  1#@param {type:"number"}
n0=[n_N2O,n_O2,n_NO2]

Calculate enthalpy from the Shoemate equation using coefficients imported from webbook. The first four terms of the Shoemate equation are just a polynomial terms. Add them first. Then add the 1/T and constant terms.
Multiply by stoichometery and add up to get the total change in enthalpy due to temperature change from 298. Calculate the standard (298 K) enthalpy and then add it to the temperature change enthalpy for the enthalpy change at $T$.  If we consider ideal gases here, here will be no correction for a change in pressure from 1 bar.

In [69]:
Hf=[sum([coef[i][j]/(j+1)*(T/1000)**(j+1) for j in range(4)]) for i in range(len(stoic))]
Hf=[Hf[i]-coef[i][4]*(1000/T)+coef[i][5] for i in range(len(stoic))] # temperature change for each species. This is the enthalpy of formation of each species at Tf.
DH=sum([Hf[i]*stoic[i] for i in range(len(stoic))])# total enthalpy change at Tf added up using stoichiometry for each species.

Hfo=sum([coef[i][7]*stoic[i] for i in range(len(stoic))]) # standard reaction enthalpy change. The 7th element of coeff is the standard enthalpy.

DHtot=DH+Hfo #add temperature change enthalpy to standard enthalpy

print('standard enthalpy= {:3.3f} kJ/mol      enthalpy at {:3d} K= {:3.3f} kJ/mol'.format(Hfo,Tf,DH.subs(T,Tf).round(10))) 
print('enthalpies of formation at 298 K:     ',[round(coef[i][7],2) for i in range(len(stoic))])
print('enthalpies of formation at',Tf,'K:     ',[Hf[i].subs(T,Tf).round(2) for i in range(len(stoic))])
print('difference:                           ',[round(Hf[i].subs(T,Tf)-coef[i][7],2) for i in range(len(stoic))])

standard enthalpy= -31.716 kJ/mol      enthalpy at 450 K= -34.095 kJ/mol
enthalpies of formation at 298 K:      [82.05, 0.0, 33.1]
enthalpies of formation at 450 K:      [88.37, 4.54, 39.07]
difference:                            [6.33, 4.54, 5.98]


Do a similar calculation for entropy. The difference here is that entropy is absolute so we don't need to add onto the entropy at 298 K. We could adjust for the effect of change in pressure here or do it in the $\Delta G$ calculation later. We'll do it both ways.

In [97]:
S=[sum([coef[i][j]/(j)*(T/1000)**j for j in range(4) if j>0]) for i in range(len(stoic))] # take care of the polynomial part of the equation
S=[S[i]+coef[i][0]*ln(T/1000)-coef[i][4]/(2*(T/1000)**2)+coef[i][6] for i in range(len(stoic))] # add terms for the non-polynomial terms
DStot=sum([S[i]*stoic[i] for i in range(len(stoic))]) # add up the entropies for all species with the appropriate stoichiometry
DStot_p=DStot-sum(stoic)*R*ln(pf) #If we account for pressure here. Below for pressure accounted for in the delta G equation.
print('total change in entropy at {:3d} K: {:3.3f} J/mol K'.format(Ti,DStot.subs(T,298).evalf()))
print('total change in entropy at {:3d} K: {:3.3f} J/mol K\ntotal change in entropy at {:3d} K and {:2.1f} bar: {:3.3f}'.format(Tf,DStot.subs(T,Tf).evalf(),Tf, pf,DStot_p.subs(T,Tf).evalf()))
print('Delta G from Delta H and Delta S at {:3d} K and {:2.1f} bar: {:3.3f} J/mol'.format(Tf,pi,(DHtot*1000-Tf*DStot).subs(T,Tf).evalf()))
print('Delta G from Delta H and Delta S at {:3d} K and {:2.1f} bar: {:3.3f} J/mol'.format(Tf,pf,(DHtot*1000-Tf*DStot_p).subs(T,Tf).evalf()))


total change in entropy at 298 K: -95.204 J/mol K
total change in entropy at 450 K: -101.728 J/mol K
total change in entropy at 450 K and 0.1 bar: -120.873
Delta G from Delta H and Delta S at 450 K and 1.0 bar: -20033.404 J/mol
Delta G from Delta H and Delta S at 450 K and 0.1 bar: -11418.266 J/mol


In [105]:
#Do again using integration of Cp instead of the Shomate equation
t=symbols('t')
pow=[0,1,2,3,-2] #powers of the Shomate equation for Cp
Cpt=[sum([coef[j][i]*(t/1000)**pow[i] for i in range(len(pow)) ]) for j in range(len(stoic))]
Ht=[integrate(Cpt[i],(t,298,Tf)) for i in range(len(stoic))]
St=[integrate(Cpt[i]/t,(t,298,Tf)) for i in range(len(stoic))]
print('enthalpy change= {:3.3f} kJ/mol       \nentropy change for change from 298 K to {:3d}: {:3.3f} J/mol'.format(sum([Ht[i]*stoic[i] for i in range(len(stoic))])/1000+Hfo,Tf,(sum([St[i]*stoic[i] for i in range(len(stoic))])+DStot.subs(T,298)).evalf()))
print('entropy change for change from 1 bar to {:2.1f}: {: 3.3f} J/mol K'.format(pf,-sum(stoic)*R*ln(pf)))
print('total entropy change: {:3.3f} J/mol K'.format((sum([St[i]*stoic[i] for i in range(len(stoic))])+DStot.subs(T,298)).evalf()-sum(stoic)*R*ln(pf)))

enthalpy change= -34.100 kJ/mol       
entropy change for change from 298 K to 450: -101.728 J/mol
entropy change for change from 1 bar to 0.1: -19.145 J/mol K
total entropy change: -120.873 J/mol K


# Calculation of S, H, and G for each species
None of this is necessary but some do it this way.

In [92]:
[coef[i][7]+Hf[i].subs(T,Tf)-450*S[i].subs(T,Tf).round(2)/1000 for i in range(len(stoic))] #delta Gs for each species but using S not Delta S. This isn't right but it works if you take products minus reactants

[63.7591841838957, -93.3099484193333, -43.1109700830330]

In [95]:
print('Standard entropies: ',So) #standard entropies
print('Standard entropy change at T=',Tf,'K:', sum([So[i]*stoic[i] for i in range(len(stoic))]),'J/mol K') #standard entropy change at 298 K and 1 bar.

Standard entropies:  [219.96, 205.15, 240.04]
Standard entropy change at T= 450 K: -95.21000000000015 J/mol K


In [94]:
[print(S[i].subs(T,Tf).round(2),'J/mol K') for i in range(len(stoic))]# entropy of each species at Tf
[print(Hf[i].subs(T,Tf).round(2),'kJ/mol') for i in range(len(stoic))] #enthapy change for each species to get to Tf
[print((Hf[i]+coef[i][7]).subs(T,Tf).round(2),'kJ/mol') for i in range(len(stoic))] #enthalpy of formation at Tf

237.03 J/mol K
217.45 J/mol K
256.17 J/mol K
88.37 kJ/mol
4.54 kJ/mol
39.07 kJ/mol
170.42 kJ/mol
4.54 kJ/mol
72.17 kJ/mol


[None, None, None]

In [96]:
DG=DHtot*1000-T*DStot+sum(stoic)*R*T*log(pf)
print('Delta G at Tf=',Tf,'K=',(DHtot*1000-T*DStot).subs(T,Tf).round(2),'J/mol         \nDelta G of pressure change=', (sum(stoic)*R*T*log(pf)).subs(T,Tf).round(2),'J/mol               \ntotal Delta G=',DG.subs(T,Tf).n().round(2),'J/mol') # DG... same as above, calculated differently

Delta G at Tf= 450 K= -20033.40 J/mol         
Delta G of pressure change= 8615.14 J/mol               
total Delta G= -11418.27 J/mol


# K and moles calculations

In [ ]:
Keq=exp(-DG/R/T)
print('Keq=',Keq.subs(T,Tf)) #calculate Keq

In [ ]:
moles=[n0[i]+stoic[i]*xi for i in range(len(stoic))]; #calculate moles with xi 
molesum=sum(moles)                                    # and sum them up.
chi=[(n0[i]+stoic[i]*xi)/sum(moles) for i in range(len(stoic))]

In [ ]:
ximin=max([-n0[i]/stoic[i] for i in range(len(stoic)) if stoic[i]>0]) #calculate the high end of the xi range
ximax=min([-n0[i]/stoic[i] for i in range(len(stoic)) if stoic[i]<0]) #calculate the low end of the xi range

print('xi is between',ximin,'and',ximax)

Now we define a function that can be solved for $\xi$. The function needs to be equal to zero so we'll set it up so that $\frac{\Pi_i [products]_i^{a_i}}{\Pi_j[reactants]_j^{a_j}}-K_{eq}=0$, where $a$ is the stoichiometric coefficient for species $i$ or $j$. But really it's easier to solve if we take away the fraction and write it as $\Pi_i[products]_i^{a_i}-K_{eq}\times\Pi_j[reactants]_j^{a_j}=0$.

Because of this form, it is conventient to deal with the numerator and the denominator of the equilibrium constant separately. Each of these is just a product of each of the products or each of the reactants all to their appropriate powers. There is no $product$ function in most now current python versions but 'reduce(lambda x,y:x*y,list)' accomplishes this task for the lists of terms in the numerator ($num$) and the denominator ($den$). 

In [ ]:
def K(x):
  num=[chi[i].subs(xi,x)**stoic[i] for i in range(len(stoic)) if stoic[i]>0]
  den=[chi[i].subs(xi,x)**-stoic[i] for i in range(len(stoic)) if stoic[i]<0]
  return reduce(lambda x,y:x*y,num)-Keq.subs(T,Tf)*reduce(lambda x,y:x*y,den) # 

Now we use one of the equation solvers, here $brentq$, to solve our function between $\xi_{min}$ and $\xi_{max}$.

In [ ]:
x=brentq(K,ximin,ximax) #find zeros of the function above between the minimum and maximum possible xi.
round(x,4)

Now we can use $\xi$ to calculate the number of moles of each reactant and product at equilibrium.

In [ ]:
n=[moles[i].subs(xi,x) for i in range(len(stoic))] # substitute xi into moles expressions to get answer.
output=[print(species[i],':',n[i].round(2),"mol") for i in range(len(stoic))]

A test for consistency is always good. A calculation of the difference in $K_{eq}$ using moles and using $\Delta G$ should give zero. Here we allow it to be as big as $\pm10^{-8}$. 

In [ ]:
kk=[chi[i].subs(T,Tf).subs(xi,x)**stoic[i] for i in range(len(stoic))] #test equilibrium constant in terms of xi
print(abs(reduce(lambda x,y:x*y,kk)-Keq.subs(T,Tf))<10**-8) #make sure the K calculated from moles matches the K calculated from Delta G (or close enough). True means we're good.